In [1]:
import pandas as pd
import numpy as np
import rasterio



In [2]:
import h3


def h3_to_ij(h3_index, hex_res):
    # Convert h3 index to lat/lon coordinates
    lat, lon = h3.h3_to_geo(h3_index)

    # Calculate x and y coordinates of hexagon center
    x = (lon + 180) / 360 * (2 ** hex_res)
    y = ((lat * -1) + 90) / 180 * (2 ** hex_res)

    # Calculate i and j array indices
    i = int(y)
    j = int(x)

    return i, j


In [21]:
# Create sample DataFrame with hex7 and cnt attributes
df = pd.DataFrame({
    'hex7': ['873082490ffffff', '873082491ffffff', '873082492ffffff'],
    'cnt': [10, 20, 30]
})

# Define hex resolution
hex_res = 7

# Create hexagon grid and populate with cnt values
hex_grid = np.zeros((122, 244))
for index, row in df.iterrows():
    h3_index = row['hex7']
    cnt_value = row['cnt']
    i, j = h3_to_ij(h3_index, hex_res)
    hex_grid[i][j] = cnt_value

# Define output file name
output_file = "output.tiff"

# Define metadata for output file
width = 1220
height = 1220
count = 3
dtype = hex_grid.dtype
transform = rasterio.transform.Affine(1, 0, 0, 0, -1, height)

# Write NumPy array to TIFF file
with rasterio.open(output_file, 'w', driver='GTiff', width=width, height=height, count=count, dtype=dtype, transform=transform) as dst:
    dst.write(hex_grid, 1)


In [19]:
import h3
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio



# Define the H3 resolution and the output file name
h3_resolution = 7
output_file = "output.tiff"

# Create sample DataFrame with hex7 and cnt attributes
df = pd.DataFrame({
    'hex7': ['873082490ffffff', '873082491ffffff', '873082492ffffff'],
    'cnt': [10, 20, 30]
})

# Convert the H3 hexagons to GeoDataFrames
polygons = []
for index, row in df.iterrows():
    h3_index = row['hex7']
    cnt_value = row['cnt']
    hex_boundary = h3.h3_to_geo_boundary(h3_index)
    poly = Polygon(hex_boundary)
    polygons.append(poly)

gdf = gpd.GeoDataFrame(df, geometry=polygons)

gdf = gdf.set_crs(epsg=4326)

# Define the target CRS and transform the polygons
target_crs = 'EPSG:4326'
gdf = gdf.to_crs(target_crs)

# Get the envelope of the polygons
envelope = gdf.envelope

# Get the bounds of the envelope
xmin, ymin, xmax, ymax = envelope.total_bounds

# Define the metadata for the output file
pixel_size = 0.1  # Resolution of the output file
nx = int((xmax - xmin) / pixel_size)  # Number of pixels in x direction
ny = int((ymax - ymin) / pixel_size)  # Number of pixels in y direction
dtype = rasterio.float32  # Data type of the output file
count = 1  # Number of bands in the output file
transform = rasterio.transform.from_bounds(xmin, ymin, xmax, ymax, nx, ny)

# Write the GeoDataFrame to a TIFF file
with rasterio.open(output_file, 'w', driver='GTiff', width=nx, height=ny, count=count, dtype=dtype, transform=transform) as dst:
    dst.write(gdf.values, 1)


OSError: exception: access violation writing 0x0000018685238990